In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('ukTrafficAADF.csv')

In [3]:
income = pd.read_excel('vcregionalgdhibylareordered.xlsx', sheet_name='GDHI per head', 
                       skiprows = [0,1,], nrows=391)
income.drop('LAU1 code', axis=1, inplace=True)
population = pd.read_excel('vcregionalgdhibylareordered.xlsx', sheet_name='Population',
                          skiprows = [0,1,])
population.drop('LAU1 code', axis=1, inplace=True)

In [4]:
population.head()

,Region,LA name,1997,1998,1999,2000,2001,2002,2003,2004,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,North East,County Durham,498533,496945,496096,495138,493678,493538,493931,495147,...,502960,505617,507311,510628,512994,514261,515923,517573,519347,521776
1,North East,Darlington,99171,99273,98172,98088,97894,98474,98911,99343,...,102632,103694,104355,105028,105584,105503,105726,105877,105998,106327
2,North East,Hartlepool,89921,89753,89680,89811,90152,89993,90134,90317,...,90969,91379,91530,91773,92088,92261,92662,92606,92498,92845
3,North East,Middlesbrough,144497,144031,143023,142251,141233,140090,139050,138451,...,138190,137885,137273,137667,138368,138726,138911,138991,139310,140326
4,North East,Northumberland,306756,307241,307490,307348,307363,308499,309448,310140,...,312960,314057,314489,315463,316278,316489,316389,316832,316453,317444


In [5]:
income_piv = income.melt(id_vars=['Region', 'LA name'], var_name='Year', value_name='Gross Disposable Household Income')
pop_piv = population.melt(id_vars=['Region', 'LA name'], var_name='Year', value_name='Population')

In [14]:
cols = ['Region', 'LA name']

income_piv[cols] = income_piv[cols].apply(lambda x: x.str.lower(), axis=0)
pop_piv[cols] = pop_piv[cols].apply(lambda x: x.str.lower(), axis=0)
data2 = pd.concat([income_piv.set_index(income_piv.columns[:3].tolist()),
                   pop_piv.set_index(pop_piv.columns[:3].tolist())], axis=1).reset_index()
data2.columns = ['Region', 'LocalAuthority', 'AADFYear', 'Gross Disposable Household Income', 'Population']

In [15]:
data2.head()

,Region,LocalAuthority,AADFYear,Gross Disposable Household Income,Population
0,north east,county durham,1997,9238,498533
1,north east,darlington,1997,9637,99171
2,north east,hartlepool,1997,8963,89921
3,north east,middlesbrough,1997,8909,144497
4,north east,northumberland,1997,10301,306756


In [8]:
cols = ['Region', 'LocalAuthority']
for col in cols:
    data[col] = data[col].str.lower()

In [9]:
df = data.merge(data2, how='left', on=['Region', 'LocalAuthority', 'AADFYear'])

In [10]:
df.head()

,AADFYear,CP,Estimation_method,Estimation_method_detailed,Region,LocalAuthority,Road,RoadCategory,Easting,Northing,...,V4or5AxleRigidHGV,V3or4AxleArticHGV,V5AxleArticHGV,V6orMoreAxleArticHGV,AllHGVs,AllMotorVehicles,Lat,Lon,Gross Disposable Household Income,Population
0,2000,6007,Counted,Manual count,yorkshire and the humber,rotherham,M1,TM,446000,389300,...,711,1588,4715,2149,15170,111146,53.398531,-1.309655,NaN,NaN
1,2000,6009,Counted,Manual count,yorkshire and the humber,leeds,M621,TM,432150,429500,...,287,628,1494,921,4895,65939,53.760869,-1.513814,NaN,NaN
2,2000,6035,Counted,Manual count,yorkshire and the humber,doncaster,M18,TM,466400,407900,...,239,745,3695,2000,8888,40229,53.563523,-0.998970,10215.0,286989.0
3,2000,6054,Counted,Manual count,yorkshire and the humber,calderdale,M62,TM,404000,416600,...,354,1786,5902,3438,15109,85759,53.645893,-1.940967,10773.0,191928.0
4,2000,6055,Counted,Manual count,yorkshire and the humber,leeds,M62,TM,426000,426200,...,423,1947,6669,3743,17436,98487,53.731551,-1.607374,NaN,NaN


In [11]:
df.LocalAuthority.unique().shape

(190,)

In [12]:
df.to_csv('AADF data with income and population.csv')

In [70]:
df = df.dropna()

In [ ]:
df.groupby(['AADFYear','Region_x'])['Gross Disposable Household Income'].mean().unstack('Region_x').plot(figsize=(10,8),kind='line')
plt.show()

In [ ]:
df.groupby(['AADFYear','Region_x'])['AllMotorVehicles'].mean().unstack('Region_x').plot(figsize=(10,8),kind='line')
plt.show()

In [ ]:
df.groupby(['AADFYear','Region_x'])[['AllMotorVehicles']].mean().unstack('Region_x').plot(figsize=(10,8),kind='line')
plt.show()

In [ ]:
df.groupby(['AADFYear','Region_x'])[['PedalCycles']].mean().unstack('Region_x').plot(figsize=(10,8),kind='line')
plt.show()

In [ ]:
df.columns

In [106]:
test = test[['date_time', 'Accident_Severity']].copy()
accidents = test.loc[test.Accident_Severity < 3].groupby(pd.Grouper(key='date_time', freq='D')).size().nlargest(5)
accidents.index.tolist()

[Timestamp('2005-05-14 00:00:00'),
 Timestamp('2005-06-18 00:00:00'),
 Timestamp('2005-09-16 00:00:00'),
 Timestamp('2005-11-04 00:00:00'),
 Timestamp('2005-12-23 00:00:00')]